In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [0]:
toremove = []
for items in train.columns:
  if items != "ID" and items != "target":
    if train[items].std() == 0:
      toremove.append(items)

train.drop(toremove, axis=1, inplace=True)
test.drop(toremove, axis=1, inplace=True)

In [0]:
def duplicates(df):
  x = df.columns.to_series().groupby(df.dtypes).groups
  duplicate = []
  
  for a, b in x.items():
    y = df[b].columns
    z = df[b]
    length = len(y)
    
    for i in range(length):
      g = z.iloc[:, i].values
      for h in range(i+1, length):
        j = z.iloc[:, h].values
        if np.array_equal(g, j):
          duplicate.append(y[i])
          break
  
  return duplicate

toremove = duplicates(train)

In [0]:
train.drop(toremove, axis=1, inplace=True)

test.drop(toremove, axis=1, inplace=True)

In [0]:
def sparse_remove(df1, df2):
  list1 = [x for x in df1.columns if not x in ["ID", "target"]]
  for items in list1:
    if len(np.unique(df1[items])) < 2:
      df1.drop(items, axis=1, inplace=True)
      df2.drop(items, axis=1, inplace=True)
  return df1, df2

train, test = sparse_remove(train, test)

In [0]:
X_train = train.drop(["ID", "target"], axis=1)
Y_train = np.log1p(train["target"].values)
x_test = test.drop(["ID"], axis=1)

In [0]:
devel_X, val_X, devel_y, val_y = train_test_split(X_train, Y_train, test_size = 0.2, random_state=44)

In [0]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
  params = {
      "objective": "regression",
      "metric": "rmse",
      "num_leaves": 42,
      "learning_rate": .01,
      "bagging_fraction": .75,
      "feature_fraction": .75,
      "bagging_frequency": 8,
      "bagging_seed": 42,
      "verbosity": -1,
      "seed": 42
  }
  
  lgtrain = lgb.Dataset(train_X, label=train_y)
  lgval = lgb.Dataset(val_X, label=val_y)
  eval_result = {}
  model = lgb.train(params, lgtrain, 5000, valid_sets=[lgtrain, lgval],
                   early_stopping_rounds = 120, verbose_eval = 150, 
                   evals_result = eval_result)
  pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
  return pred_test_y, model, eval_result

In [59]:
pred_test, model, eval_result = run_lgb(devel_X, devel_y, val_X, val_y, x_test)

Training until validation scores don't improve for 120 rounds.
[150]	training's rmse: 1.27164	valid_1's rmse: 1.44967
[300]	training's rmse: 1.06447	valid_1's rmse: 1.40813
[450]	training's rmse: 0.945947	valid_1's rmse: 1.40421
Early stopping, best iteration is:
[467]	training's rmse: 0.935491	valid_1's rmse: 1.40417
